* Which year was the largest production ever made, narrow it down to the month with the highest?
* Which year has the highest average production for our records, and which year has the highest average price.
* Is there a month where the total production was exported totally?
* How many times was oil exported less than 70% of the production.
* View of production and prices in three years difference
* Summarize the three numeric column
* What is the correlation of price with production and export?
* Summarize for the last five years, the average price, standard deviation and median. 

In [54]:
import sqlite3 as sq
import plotly.graph_objs as go
import plotly.express as px

In [2]:
conn = sq.connect("crude_oil.db")

In [3]:
# Create table called crudeOilPrice
query = """
CREATE TABLE crudeOilPrice(
    year VARCHAR(4),
    month SMALLINT,
    crude_oil_price DECIMAL(5,2),
    production DECIMAL(3,2),
    crude_oil_export DECIMAL(3,2)
    )"""

with conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS crudeOilPrice")
    cur.execute(query)

In [4]:
# checking our sqlite master for confirmation
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
print(cur.fetchall())

[('crudeOilPrice',)]


In [5]:
# previewing the table content
query = "SELECT * FROM crudeOilPrice"
cur.execute(query)
print(cur.fetchall())

[]


In [6]:
# coverting sql table to dataframe
import pandas as pd

sqlDF = pd.read_sql_query(query, conn)
sqlDF

,year,month,crude_oil_price,production,crude_oil_export


Let fill in values from the csv


In [7]:
# Loading CSV file
csv = pd.read_csv('CrudeOilProdPrice25102022.csv', index_col=False)
csv.head()

,Year,Month,Crude Oil Price,Production,Crude Oil Export
0,2006,1,63.85,2.59,2.14
1,2006,2,61.33,2.47,2.02
2,2006,3,65.00,2.25,1.80
3,2006,4,72.09,2.32,1.87
4,2006,5,71.18,2.28,1.83


In [8]:
# creating a tuple of values in each row
csvTuple = tuple(csv.itertuples(index=False, name=None))
csvTuple[0]

(2006, 1, 63.85, 2.59, 2.14)

In [9]:
# inserting into table on the database
insertQuery = "INSERT INTO crudeOilPrice values(?,?,?,?,?)"
cur.executemany(insertQuery, csvTuple)

In [10]:
# Previewing table
query = "SELECT * FROM crudeOilPrice"

# Reading table as a pandas dataframe
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,year,month,crude_oil_price,production,crude_oil_export
0,2006,1,63.85,2.59,2.14
1,2006,2,61.33,2.47,2.02
2,2006,3,65.00,2.25,1.80
3,2006,4,72.09,2.32,1.87
4,2006,5,71.18,2.28,1.83
...,...,...,...,...,...
196,2022,5,116.72,1.02,0.57
197,2022,6,130.10,1.16,0.71
198,2022,7,120.54,1.08,0.63
199,2022,8,106.34,0.97,0.52


## Data Cleaning

In [11]:

query = """
UPDATE
    crudeOilPrice
SET 
    month = 
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(month, 12, 'December'), 
        2, 'February'), 
        3, 'March'),
        4, 'April'),
        5, 'May'), 
        6, 'June'),
        7, 'July'),
        8, 'August'),
        9, 'September'),
        10, 'October'),
        11, 'November'),
        1, 'January')
"""

cur.execute(query)

In [12]:
query = "SELECT * FROM crudeOilPrice"
pd.read_sql_query(query, conn)

,year,month,crude_oil_price,production,crude_oil_export
0,2006,January,63.85,2.59,2.14
1,2006,February,61.33,2.47,2.02
2,2006,March,65.00,2.25,1.80
3,2006,April,72.09,2.32,1.87
4,2006,May,71.18,2.28,1.83
...,...,...,...,...,...
196,2022,May,116.72,1.02,0.57
197,2022,June,130.10,1.16,0.71
198,2022,July,120.54,1.08,0.63
199,2022,August,106.34,0.97,0.52


## Exploration

In [14]:
# Overview of record
query = """
SELECT COUNT(DISTINCT year) AS Years_record,
       MIN(year) AS Start_year,
       MAX(year) AS End_year
  FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,Years_record,Start_year,End_year
0,17,2006,2022


* Which year was the largest production ever made, narrow it down to the month?

In [15]:
query = """
SELECT  year, month, max(production) AS Max_Prod
  FROM  crudeOilPrice
"""
pd.read_sql_query(query, conn)

,year,month,Max_Prod
0,2010,October,2.88


* Which year has the highest average production for our record.


In [16]:
query = """
SELECT sub.Year, max(sub.Avg_Prod) AS Max_Avg_Prod
  FROM (
          SELECT year AS Year, avg(production) AS Avg_Prod
            FROM crudeOilPrice
        GROUP BY Year) AS sub
"""
pd.read_sql_query(query, conn)

,Year,Max_Avg_Prod
0,2010,2.4675


 * Which year has the highest average price.

In [17]:
query = """
SELECT sub.Year, max(Avg_Price) AS Max_Avg_Price
  FROM (
          SELECT year AS Year, avg(crude_oil_price) AS Avg_Price
            FROM crudeOilPrice
        GROUP BY Year) AS sub
"""
pd.read_sql_query(query, conn)

,Year,Max_Avg_Price
0,2011,113.76


Is there a month where the total production was exported totally?

In [18]:
query = """
SELECT COUNT(*) AS count
  FROM crudeOilPrice
 WHERE production = crude_oil_export
"""
pd.read_sql_query(query, conn)

,count
0,0


How many times was oil exported less than 70% of the production.

In [19]:
query = """
SELECT  COUNT(*) AS Count
  FROM  (
        SELECT production, year, crude_oil_export, 0.7 * production AS fifty_perc_prod
  FROM  crudeOilPrice)
 WHERE  crude_oil_export < fifty_perc_prod
"""
pd.read_sql_query(query, conn)

,Count
0,22


We have 22 count, let's narrow it down to year with the highest count.

In [21]:
query = """
SELECT production, year, crude_oil_export, 0.7 * production AS fifty_perc_prod
  FROM crudeOilPrice
 LIMIT 10
"""
pd.read_sql_query(query, conn)

,production,year,crude_oil_export,fifty_perc_prod
0,2.59,2006,2.14,1.813
1,2.47,2006,2.02,1.729
2,2.25,2006,1.80,1.575
3,2.32,2006,1.87,1.624
4,2.28,2006,1.83,1.596
5,2.41,2006,1.96,1.687
6,2.39,2006,1.94,1.673
7,2.40,2006,1.95,1.680
8,2.40,2006,1.95,1.680
9,2.38,2006,1.93,1.666


In [20]:
query = """
  SELECT year AS Year, COUNT(*) AS Export_Count
    FROM (
           SELECT production, year, crude_oil_export, 0.7 * production AS fifty_perc_prod
             FROM crudeOilPrice)
   WHERE crude_oil_export < fifty_perc_prod
GROUP BY Year
"""
pd.read_sql_query(query, conn)

,Year,Export_Count
0,2020,1
1,2021,12
2,2022,9


We have seen the years that exported oil was below 70% of production. 

Let's look at when exported oil was above 80%

In [49]:
query = """
  SELECT year AS Year, COUNT(*) AS Export_Count
    FROM (
          SELECT production, year, crude_oil_export, 0.8 * production AS fifty_perc_prod
          FROM crudeOilPrice)
   WHERE crude_oil_export >= fifty_perc_prod
GROUP BY Year
"""
sqlDF = pd.read_sql_query(query, conn, index_col='Year')
sqlDF

,Export_Count
Year,
2006,12
2007,5
2008,1
2009,1
2010,11
2011,12
2012,8
2013,2
2014,3


In [58]:
fig = px.line(sqlDF, x=sqlDF.index, y=sqlDF.Export_Count, 
                markers=True, text='Export_Count', width=900,
                title='Crude oil eport above 80% of production',)
                # labels={'day_period':'', 'order_count':'Order Count'})
fig

In [62]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sqlDF.index, y=sqlDF.Export_Count, text='Export_Count',
                         name='2021 Production', line=dict(color='firebrick', 
                                                           width=4, dash='dot'), markers=True))
fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'markers'

Did you mean "marker"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        dx
            Sets the x coordinate

It turns out that the last time oil was exported above 80% of production was in 2014. And its only in 2006 and 2011 that 80% of produced oil was exported in all months.

In [23]:
query = """
    SELECT  c1.month, c1.crude_oil_price AS Price2006, c2.crude_oil_price AS Price2011
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2006 AND c2.year = c1.year + 5
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,month,Price2006,Price2011
0,January,63.85,97.96
1,February,61.33,106.57
2,March,65.00,116.56
3,April,72.09,124.49
4,May,71.18,118.43
5,June,69.32,117.03
6,July,75.13,117.86
7,August,75.15,111.99
8,September,62.97,115.73
9,October,59.49,113.12


View of price in three years gap

In [24]:
query = """
WITH first AS (
    SELECT  c1.month, c1.crude_oil_price AS Price2006, c2.crude_oil_price AS Price2009
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2006 AND c2.year = c1.year + 3),

sec AS (
    SELECT  c1.month, c1.crude_oil_price AS Price2012, c2.crude_oil_price AS Price2015
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2012 AND c2.year = c1.year + 3),

third AS (
    SELECT  c1.month, c1.crude_oil_price AS Price2018, c2.crude_oil_price AS Price2021
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2018 AND c2.year = c1.year + 3)
     
    SELECT  f.month, f.Price2006, f.Price2009, s.Price2012, s.Price2015, t.Price2018, t.Price2021
      FROM  first AS f
 LEFT JOIN  sec AS s
     USING  (month)
 LEFT JOIN  third AS t
     USING  (month)
"""

pd.read_sql_query(query, conn)

,month,Price2006,Price2009,Price2012,Price2015,Price2018,Price2021
0,January,63.85,44.95,113.81,48.81,69.68,54.87
1,February,61.33,46.52,121.87,58.09,66.67,62.48
2,March,65.00,49.70,128.00,56.69,74.72,65.62
3,April,72.09,51.16,122.62,57.45,72.37,64.30
4,May,71.18,60.02,113.08,65.08,77.64,67.83
5,June,69.32,72.24,98.06,62.06,75.38,73.46
6,July,75.13,66.52,104.62,57.01,74.72,75.93
7,August,75.15,74.00,113.76,47.09,73.35,70.72
8,September,62.97,70.22,114.36,48.08,79.59,74.55
9,October,59.49,78.25,108.92,48.86,79.18,84.11


In [25]:
#preview year details
query = "SELECT * FROM crudeOilPrice WHERE year = '2021'"
pd.read_sql_query(query, conn)

,year,month,crude_oil_price,production,crude_oil_export
0,2021,January,54.87,1.36,0.91
1,2021,February,62.48,1.42,0.97
2,2021,March,65.62,1.43,0.98
3,2021,April,64.30,1.37,0.92
4,2021,May,67.83,1.34,0.89
5,2021,June,73.46,1.31,0.86
6,2021,July,75.93,1.32,0.87
7,2021,August,70.72,1.24,0.79
8,2021,September,74.55,1.25,0.80
9,2021,October,84.11,1.23,0.78


* Summarize the three numeric column

In [26]:
query = """
SELECT  min(crude_oil_price) Min_Price,
        avg(crude_oil_price) Avg_Price,
        max(crude_oil_price) Max_Price
  FROM  crudeOilPrice
"""

pd.read_sql_query(query, conn)

,Min_Price,Avg_Price,Max_Price
0,14.28,78.167463,138.74


Summary of Oil Price, Production and Export By Month and Year

In [27]:
query = """
  SELECT month Month,
         min(crude_oil_price) Min_Price,
         avg(crude_oil_price) Avg_Price,
         max(crude_oil_price) Max_Price
    FROM crudeOilPrice
GROUP BY month
ORDER BY Avg_Price
"""

pd.read_sql_query(query, conn)

,Month,Min_Price,Avg_Price,Max_Price
0,January,30.66,73.427059,115.24
1,December,37.80,73.533750,114.49
2,November,42.70,75.022500,113.92
3,February,31.70,75.730588,121.87
4,October,39.74,76.221875,113.12
5,March,32.29,78.700588,128.00
6,April,14.28,78.901176,124.49
7,September,40.85,79.057647,115.73
8,August,45.06,79.690588,115.84
9,May,27.90,81.265882,126.57


In [28]:
query = """
  SELECT year Year,
         min(crude_oil_price) Min_Price,
         avg(crude_oil_price) Avg_Price,
         max(crude_oil_price) Max_Price
    FROM crudeOilPrice
GROUP BY year
ORDER BY Avg_Price
"""

pd.read_sql_query(query, conn)

,Year,Min_Price,Avg_Price,Max_Price
0,2020,14.28,41.890000,66.68
1,2016,30.66,43.806667,53.48
2,2015,37.80,52.653333,65.08
3,2017,46.39,54.085833,65.11
4,2009,44.95,63.900000,78.25
5,2019,59.10,65.850000,73.65
6,2006,59.49,66.668333,75.15
7,2021,54.87,70.120000,84.11
8,2018,62.00,72.657500,79.59
9,2007,55.57,74.962500,95.05


In [29]:
query = """
  SELECT month Month,
         min(production) Min_Production,
         avg(production) Avg_Production,
         max(production) Max_Production
    FROM crudeOilPrice
GROUP BY month
ORDER BY Avg_Production
"""

pd.read_sql_query(query, conn)

,Month,Min_Production,Avg_Production,Max_Production
0,June,1.16,1.971176,2.41
1,May,1.02,1.978824,2.50
2,December,1.20,2.009375,2.58
3,July,1.08,2.009412,2.48
4,April,1.22,2.014118,2.42
5,March,1.24,2.015294,2.44
6,August,0.97,2.015882,2.50
7,September,0.94,2.021176,2.48
8,November,1.23,2.035625,2.50
9,February,1.26,2.095294,2.51


In [30]:
query = """
  SELECT year Year,
         min(production) Min_Production,
         avg(production) Avg_Production,
         max(production) Max_Production
    FROM crudeOilPrice
GROUP BY year
ORDER BY Avg_Production
"""

pd.read_sql_query(query, conn)

,Year,Min_Production,Avg_Production,Max_Production
0,2022,0.94,1.143333,1.40
1,2021,1.20,1.308333,1.43
2,2020,1.42,1.755833,2.07
3,2016,1.50,1.816667,2.15
4,2017,1.60,1.889167,2.01
5,2018,1.78,1.915000,2.01
6,2019,1.94,2.012500,2.11
7,2008,1.96,2.099167,2.26
8,2009,1.86,2.110833,2.28
9,2015,1.97,2.126667,2.22


In [31]:
query = """
  SELECT month Month,
         min(crude_oil_export) Min_Export,
         avg(crude_oil_export) Avg_Export,
         max(crude_oil_export) Max_Export
    FROM crudeOilPrice
GROUP BY month
ORDER BY Avg_Export
"""

pd.read_sql_query(query, conn)

,Month,Min_Export,Avg_Export,Max_Export
0,June,0.71,1.521176,1.96
1,May,0.57,1.528824,2.05
2,December,0.75,1.559375,2.13
3,July,0.63,1.559412,2.03
4,April,0.77,1.564118,1.97
5,March,0.79,1.565294,1.99
6,August,0.52,1.565882,2.05
7,September,0.49,1.571176,2.03
8,November,0.78,1.585625,2.05
9,February,0.81,1.645294,2.06


In [32]:
query = """
  SELECT year Year,
         min(crude_oil_Export) Min_Export,
         avg(crude_oil_Export) Avg_Export,
         max(crude_oil_Export) Max_Export
    FROM crudeOilPrice
GROUP BY year
ORDER BY Avg_Export
"""

pd.read_sql_query(query, conn)

,Year,Min_Export,Avg_Export,Max_Export
0,2022,0.49,0.693333,0.95
1,2021,0.75,0.858333,0.98
2,2020,0.97,1.305833,1.62
3,2016,1.05,1.366667,1.70
4,2017,1.15,1.439167,1.56
5,2018,1.33,1.465000,1.56
6,2019,1.49,1.562500,1.66
7,2008,1.51,1.649167,1.81
8,2009,1.41,1.660833,1.83
9,2015,1.52,1.676667,1.77


Let's look at years that their average is above the overall average for price, production and export

In [33]:
query = """
SELECT Year, Year_avg, sub.Overall_avg
  FROM (SELECT year Year, avg(crude_oil_export) Year_avg, 
               (SELECT avg(crude_oil_export) 
                  FROM crudeOilPrice) Overall_avg
          FROM crudeOilPrice
      GROUP BY year) sub
 WHERE Year_avg > sub.Overall_avg
"""
pd.read_sql_query(query, conn)

,Year,Year_avg,Overall_avg
0,2006,1.932500,1.581741
1,2007,1.751667,1.581741
2,2008,1.649167,1.581741
3,2009,1.660833,1.581741
4,2010,2.017500,1.581741
5,2011,1.930833,1.581741
6,2012,1.868333,1.581741
7,2013,1.733333,1.581741
8,2014,1.755833,1.581741
9,2015,1.676667,1.581741


Let's see the relationship between Nigeria Crude Oil export and other attributes

Since SQLite doesn't have a built in correlation function and doen't support windows function. I will use pandas to find the relation.

In [34]:
query = "SELECT * FROM crudeOilPrice"
sqlDF = pd.read_sql_query(query, conn)
sqlDF.corr()["crude_oil_export"]

crude_oil_price     0.199156
production          1.000000
crude_oil_export    1.000000
Name: crude_oil_export, dtype: float64

That's interesting! Crude oil price has a perfect positive correlation. Which means we can use just the production column and traiin a model to predict future export of crude oil.

If you want to see this model, do well to follow me as i will be taking my analysis a bit further by build a linear regression model that can predict Nigeria Future Crude Oil Export.

In [36]:
month_order = ['January', 'February', 'March', 'April', 
               'May', 'June', 'July', 'August', 'September']

query = """
SELECT  month, production AS "total_production_(mbd)", 
        crude_oil_price AS "total_price_(US$/Barrel)"
  FROM  crudeOilPrice
 WHERE  year = '2022'
"""
sqlDF = pd.read_sql_query(query, conn, index_col='month').reindex(month_order)
sqlDF

,total_production_(mbd),total_price_(US$/Barrel)
month,,
January,1.40,88.71
February,1.26,99.64
March,1.24,121.23
April,1.22,106.51
May,1.02,116.72
June,1.16,130.10
July,1.08,120.54
August,0.97,106.34
September,0.94,93.25


In [37]:
query = """
  SELECT  month, SUM(production) AS total_production, year
    FROM  crudeOilPrice
   WHERE  year = '2022'
GROUP BY  month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

       month  total_production  year
0      April              1.22  2022
1     August              0.97  2022
2   February              1.26  2022
3    January              1.40  2022
4       July              1.08  2022
5       June              1.16  2022
6      March              1.24  2022
7        May              1.02  2022
8  September              0.94  2022


In [38]:
query = """
  SELECT month, SUM(production) AS total_production, year
    FROM crudeOilPrice
   WHERE year = '2021'
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

        month  total_production  year
0       April              1.37  2021
1      August              1.24  2021
2    December              1.20  2021
3    February              1.42  2021
4     January              1.36  2021
5        July              1.32  2021
6        June              1.31  2021
7       March              1.43  2021
8         May              1.34  2021
9    November              1.23  2021
10    October              1.23  2021
11  September              1.25  2021


In [41]:
# Filtering monthly production
query = """
  SELECT c1.month, SUM(c1.production) AS total_prod_2021, sub.total_prod_2022
    FROM crudeOilPrice AS c1,
                        ( SELECT month, SUM(production) AS total_prod_2022
                            FROM crudeOilPrice
                           WHERE year = '2022'
                        GROUP BY month) AS sub
   WHERE year = '2021' AND (c1.month = sub.month) 
GROUP BY c1.month
"""
sqlDF = pd.read_sql_query(query, conn, index_col='month').reindex(month_order)
print(sqlDF)

           total_prod_2021  total_prod_2022
month                                      
January               1.36             1.40
February              1.42             1.26
March                 1.43             1.24
April                 1.37             1.22
May                   1.34             1.02
June                  1.31             1.16
July                  1.32             1.08
August                1.24             0.97
September             1.25             0.94


In [42]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sqlDF.index, y=sqlDF.total_prod_2021,
                         name='2021 Production', line=dict(color='firebrick', 
                                                           width=4, dash='dot')))
fig.add_trace(go.Scatter(x=sqlDF.index, y=sqlDF.total_prod_2022,
                         name='2022 Production', line=dict(color='royalblue', 
                                                           width=4, dash='dot')))
fig.update_layout(title='Crude Oil Production (mbd)',
                  yaxis_title = 'Millions Barrels Per Day (mbd)',
                  )
fig.show()

In [45]:
# jupyter nbconvert Crude_oil_analysis.ipynb --to slides --post serve --no-input --no-prompt